## Exercise 50 - SparkSQL
- Input:
    - A csv file containing a list of profiles
    - Header: name,surname,age
    - Each line of the file contains one profile
- name,surname,age
- Output:
    - A csv file containing one single column called “name_surname” of type String
    - name_surname = name+" "+surname


- Input:
    - name,surname,age
    - Paolo,Garza,42
    - Luca,Boccia,41
    - Maura,Bianchi,16
- Expected output:
    - name_surname
    - Paolo Garza
    - Luca Boccia
    - Maura Bianchi

In [1]:
from pyspark.sql import SparkSession

# Create a spark session
spark = SparkSession.builder.getOrCreate()

# Create a DataFrame from persons_age_name_gender.csv
dfPersons = spark.read.load('/data/students/bigdata-01QYD/ex_data/Ex49/data/',\
                            format='csv',\
                            header=True,\
                            inferSchema=True)

In [2]:
dfPersons.printSchema()
dfPersons.show()

root
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- age: integer (nullable = true)

+-----+-------+---+
| name|surname|age|
+-----+-------+---+
|Paolo|  Garza| 42|
| Luca| Boccia| 41|
|Maura|Bianchi| 16|
+-----+-------+---+



In [5]:
spark.udf.register("nameSurname",\
                  lambda name,surname: name+" "+surname)

<function __main__.<lambda>(name, surname)>

In [9]:
# Define a DF 
nameSurname = dfPersons.selectExpr("name", "surname", "nameSurname(name,surname) as name_surname")\
                .select("name_surname")

#or in SQL

# I replace DF name with a easier one
dfPersons.createOrReplaceTempView('profiles') 

nameSurnameSQL = spark.sql("""SELECT nameSurname(name,surname) as name_surname
                                      FROM profiles""")

In [10]:
nameSurname.printSchema()
nameSurname.show()

root
 |-- name_surname: string (nullable = true)

+-------------+
| name_surname|
+-------------+
|  Paolo Garza|
|  Luca Boccia|
|Maura Bianchi|
+-------------+

